Here I will attempt to validate new scripts that I wrote as well as collect revision history of selection of ~16 wikipedia pages

In [14]:
import sys
import pandas as pd
import os

sys.path.append('../scripts/')
import wikipedia_scripts

In [12]:
# Testing package

test_page = 'Second_moment_of_area'
path_to_data = os.path.join('../data/', '20180619_' + test_page + '/')

wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=test_page,
                                                      agent_email='williamvoje@gmail.com')

../scripts\wikipedia_scripts.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  df = pd.concat(pd.read_csv(i) for i in glob.iglob(directory + '*.csv'))


In [18]:
pd.read_csv(os.path.join(path_to_data, 'summary_' + test_page + '.csv'))

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,character_count,comment,external_link_count,heading_count,parentid,revid,timestamp,user,userid,wikilink_count,wikilinks
0,0,0,NaN,7227,Reverted edits by [[Special:Contribs/117.202.1...,2,11,846126925,846126944,2018-06-16T14:15:26Z,L293D,32436778,34,"['[[multiple integral]]', '[[physical unit|uni..."
1,1,1,NaN,7227,/* Definition */,2,11,846126757,846126925,2018-06-16T14:15:18Z,117.202.145.209,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
2,2,2,NaN,7227,/* Definition */,2,11,844292537,846126757,2018-06-16T14:13:49Z,117.202.145.209,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
3,3,3,NaN,7227,NaN,2,11,841071488,844292537,2018-06-04T00:06:44Z,2600:1700:F8B0:8490:7929:FC47:4FC3:CC5A,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
4,4,4,NaN,7360,Reverted to revision 829153934 by [[Special:Co...,2,11,831857911,841071488,2018-05-13T22:06:06Z,93,17369702,35,"['[[multiple integral]]', '[[physical unit|uni..."
5,5,5,NaN,7343,NaN,2,11,831857744,831857911,2018-03-22T14:40:40Z,24.221.55.43,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
6,6,6,NaN,7344,NaN,2,11,829153934,831857744,2018-03-22T14:39:35Z,24.221.55.43,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
7,7,7,NaN,7360,NaN,2,11,821241868,829153934,2018-03-06T23:45:43Z,100.33.240.50,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
8,8,8,NaN,7373,typo,2,11,816875944,821241868,2018-01-19T06:46:02Z,WereSpielChequers,4071608,35,"['[[multiple integral]]', '[[physical unit|uni..."
9,9,9,NaN,7372,NaN,2,11,816174417,816875944,2017-12-24T09:07:13Z,Victorindia87,13684355,35,"['[[multiple integral]]', '[[physical unit|uni..."


Looks like this package is working as I expected it too. Now to try significantly larger webpages.

In [21]:
list_of_pages = ['World_War_I',
                'American Civil War',
                'India',
                'Yemen' ,
                'Lady Gaga',
                'Donald Trump',
                'David Bowie',
                'Martin Luther King Jr.',
                'Google',
                '3M',
                'xHamster',
                'Reddit',
                'Vagina',
                'Human penis size',
                'Machine learning',
                'DNA_sequencing',
                'FIFA World Cup',
                'Cricket World Cup']


In [22]:
# generating_required_data

for page in list_of_pages:
    path_to_data = os.path.join('../data/', '20180619_' + page + '/')
    wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=page,
                                                      agent_email='williamvoje@gmail.com')

KeyError: 'comment'

In [ ]:
# This seemed to have error when it was processing 